In [16]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from tensorflow.keras.preprocessing.sequence import pad_sequences





In [17]:
df = pd.read_csv('Emotion_final.csv')

In [18]:
labels = sorted(list(set(df.Emotion)))
for i in range(len(labels)):
    df.Emotion[df.Emotion == labels[i]] = i

In [19]:
labels_map = {i: labels[i] for i in range(len(labels))}
labels_map

{0: 'anger', 1: 'fear', 2: 'happy', 3: 'love', 4: 'sadness', 5: 'surprise'}

In [20]:
X = df['Text']
y = df['Emotion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [21]:
y_train = np_utils.to_categorical(y_train, len(labels))
y_test = np_utils.to_categorical(y_test, len(labels))

In [22]:
X_train[:5]

9586     i had not yet gotten married and that coupled ...
5847     im feeling cautiously optimistic about the dir...
1681     i felt such a resonance with your words i feel...
1452     i like this so much but i feel like somehow th...
15890    i don t feel particularly passionate as i once...
Name: Text, dtype: object

In [23]:
tokenizer = Tokenizer(num_words=10000 )
tokenizer.fit_on_texts(X_train)

In [24]:
x_train = tokenizer.texts_to_sequences(X_train)
x_train = pad_sequences(x_train, maxlen=30)

In [25]:
model = Sequential()
model.add(Embedding(10000, 32, input_length=30))
model.add(LSTM(16))
model.add(Dense(len(labels), activation='softmax'))

In [26]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [27]:
x_test = tokenizer.texts_to_sequences(X_test)
x_test = pad_sequences(x_test, maxlen=30)

In [28]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=20)

Epoch 1/20
450/450 [==============================] - 10s 18ms/step - loss: 1.4031 - accuracy: 0.4415 - val_loss: 0.9650 - val_accuracy: 0.6837
Epoch 2/20
450/450 [==============================] - 8s 18ms/step - loss: 0.7042 - accuracy: 0.7441 - val_loss: 0.6796 - val_accuracy: 0.7499
Epoch 3/20
450/450 [==============================] - 8s 18ms/step - loss: 0.4284 - accuracy: 0.8623 - val_loss: 0.5436 - val_accuracy: 0.8297
Epoch 4/20
450/450 [==============================] - 9s 20ms/step - loss: 0.2414 - accuracy: 0.9288 - val_loss: 0.4397 - val_accuracy: 0.8567
Epoch 5/20
450/450 [==============================] - 8s 18ms/step - loss: 0.1538 - accuracy: 0.9569 - val_loss: 0.4859 - val_accuracy: 0.8581
Epoch 6/20
450/450 [==============================] - 8s 19ms/step - loss: 0.1136 - accuracy: 0.9682 - val_loss: 0.4455 - val_accuracy: 0.8637
Epoch 7/20
450/450 [==============================] - 9s 19ms/step - loss: 0.0908 - accuracy: 0.9729 - val_loss: 0.4754 - val_accuracy: 0.863

In [29]:
scores = model.evaluate(x_test, y_test)
print(f'Accuracy: {scores[1]}')

222/222 [==============================] - 1s 5ms/step - loss: 0.6734 - accuracy: 0.8578
Accuracy: 0.857808530330658
